# Document Summarization

## Introduction 
This demo showcases a chatbot system powered by Generative AI. Using technologies like <b>RAG, Langchain, and LLM models</b> users can ask questions in simple terms, retrieve relevant data, and receive concise answers. The approach integrates retrieval-based and generative techniques to deliver accurate, user-friendly insights from structured sources.

The following diagram illustrates the overall architecture.

<center><img src="images/header_chat_td.png" alt="architecture" /></center>

